# nbforms Demo Notebook

The nbforms package allows you to have interactive questions in Jupyter Notebooks that is designed to allow immediate usage of collected data. It requires you to have deployed an [`nbforms-server`](https://github.com/chrispyles/nbforms-server); the demo server is located at https://nbforms-demo-server.herokuapp.com.

### Setup

Before using nbforms, you must provide a config file. The default location of the config file is `./nbforms_config.py`; the cell below loads the config file for this notebook for your perusal.

In [ ]:
with open("./nbforms_config.json") as f:
    print(f.read())

### Usage

To use nbforms, create a `Form` instance. This will load the config file and ask the use to authenticate with the server, generating an API token for this notebook session.

**If a user has never authenticated before, the cell below will create their user, which they can then reuse on the server. There is no need to go to another page to create a user.** The rules for logins are:
* If there is no user with the username provided, a new user is created with the provided password. Then, a new API key is generated and returned to the `Form` instance.
* If there is a user with the username provided, the password provided is compared against the user's password. If this passes, a new API key is generated and returned to the `Form` instance.
* If the passwords do not match, then the cell will throw an `AssertionError`.

In [ ]:
import nbforms
form = nbforms.Form()

It is also possible to use a Google account to authenticate. To do this, specify an `auth` key in your `nbforms_config.json` file with the value set to `google`:

In [ ]:
with open("nbforms_config_google.json") as f:
    print(f.read()[:200])

When users sign in with Google, they will be asked to open a link to the nbforms server where they will authenticate with Google and then to copy their API key and input it here in the notebook. You can try this process below.

In [ ]:
google_auth_form = nbforms.Form("nbforms_config_google.json")

To ask a user to respond to a question, call `Form.ask`  with the question _identifier_ as its argument. The widget generated will have a "Submit" button which will send a POST request to your nbforms server that will record the user's response. `Form.ask` can accept multiple arguments and will display a widget for each identifier you pass it. If you pass no arguments, it will display all the widgets.

nbforms allows multiple choice (with one or many selections), text, and paragraph responses. An example of each is given below.

In [ ]:
form.ask("q1")

In [ ]:
form.ask("q2", "q3")

In [ ]:
form.ask()

#### Extracting Data from the Server

When retrieving the data from the server, nbforms allows you to collect the data into a datascience `Table` or a pandas `DataFrame`. To get the data from the server, use `Form.to_table` or `Form.to_df` and provide the question identifiers you would like to select. The optional `user_hashes` tells the server whether or not to include an randomly generated hash of each username in the CSV. *In order to protect user data, you cannot retrieve the usernames of users in the CSV.*

In [ ]:
form.to_table("q1", "q2", "q3", "q4", "q5")

In [ ]:
form.to_df("q1", "q4", "q5", user_hashes=True)

### Taking Attendance

nbforms can be used to take attendance in classes using rake tasks on the Heroku app. This is done by including an `attendance` key in the config file set to `true`. This will then allow students to run `Form.take_attendance` which will log their attendance on the server.

Attendance can be opened by running `rake attendance:open[NOTEBOOK_ID]` on the Heroku app, where `NOTEBOOK_ID` corresponds to the `notebook` parameter in your config file. There are two notebooks on the demo server, one which is always open and another that is always closed, on which you can take your attendance below.

In [ ]:
with open("attendance_open_config.json") as f:
    print("OPEN ATTENDANCE:")
    print(f.read()[:200])
    
print("\n")

with open("attendance_closed_config.json") as f:
    print("CLOSED ATTENDANCE:")
    print(f.read()[:200])

We instantiate them below and take attendance on each one.

In [ ]:
always_open = nbforms.Form("attendance_open_config.json")
always_open.take_attendance()

In [ ]:
always_closed = nbforms.Form("attendance_closed_config.json")
always_closed.take_attendance()

Attendance is always logged, so even if an attendance form is closed, a student will not know and this will be included in the attendance report.